<a href="https://colab.research.google.com/github/MilSur/ThorlabsPurchase/blob/main/ThorlabsPurchase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
############################################################
# PARAMETERS TO SET:
# please fill the desired output filename, VAT, EUR/CZK exchange rate
# and most importantly product list with Thorlabs product names and quantities

output_name = '2021_January_Milos'

vat = 1.21
eur = 26.2

product_list = [
    ['KB1P/M', 2],
    ['RS3M', 2],
    ['RSH4/M', 2],
    ['PF175B', 2],
    ['RS100P4/M', 2],
    ['RS2P4M', 2],
    ['RS1M', 2],
    ['RS12P4/M', 2],
    ['KB25/M', 2],
    ['XRN25P/M', 2],
    ['AE4M6M', 10],
    ['BA1/M-P5', 3],
    ['BA1S/M-P5', 3],
    ['KB75/M', 4],
    ['KBT75/M', 2],
    ['LG11', 1],
    ['PH50/M-P5', 1],
    ['ST1XY-S/M', 1],
    ['TPS4', 2],
    ['TPSM1/M', 4],    
]

############################################################

!pip install selenium
!pip install bs4
!pip install docx
!pip install docxtpl
!pip install price-parser
!pip install wget
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

import wget
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from docxtpl import DocxTemplate
from google.colab import files
from price_parser import Price

# Docs:
wget.download('https://github.com/MilSur/ThorlabsPurchase/blob/main/thorlabs_temp.docx?raw=true')
thorlabs_template = 'thorlabs_temp.docx'

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome('chromedriver', options=options)
browser.implicitly_wait(5)

products = []
for product in product_list:
    browser.get('https://www.thorlabs.com/thorproduct.cfm?partnumber='+str(product[0])+'&CurrencySelect=Euro')
    price_code = browser.find_elements_by_xpath("//tr/td/b[contains(text(), 'Price')]/../following-sibling::td")
    #price = float(price_code[0].text[:-2].replace(',', '.'))
    price = float(Price.fromstring(price_code[0].text).amount)
    title_code = browser.find_elements_by_xpath("//td[@class='PartTitle']")
    title = title_code[0].text
    title = title.replace(str(product[0])+' - ', '')
    products.append([product[0], title, product[1], price, '{:,}'.format(int(round(price*vat*eur*product[1], -1))).replace(',', ' ')])
browser.quit()
#print('Data successfully scraped!')

doc = DocxTemplate(thorlabs_template)

context = {
    'products': products,
    'vat': vat,
    'eur': eur,
    'total': int(sum([product[3]*vat*eur*product[2] for product in products]))/1000
}

doc.render(context)
doc.save(output_name+'.docx')
files.download(output_name+'.docx')
pd.DataFrame(product_list).to_csv(output_name+'_cart'+'.txt', sep ='\t', header = False, index = False)
files.download(output_name+'_cart'+'.txt')
print('\n\nNow, please click the folder icon on the left-sidebar, right-click the file {} and select Download'.format(output_name+'.docx'))

     |████████████████████████████████| 911kB 5.6MB/s 
     |████████████████████████████████| 61kB 2.9MB/s 
  Created wheel for docx: filename=docx-0.2.4-cp36-none-any.whl size=53926 sha256=982bd6311232e995bc36adb7b59b90625b938fddfeb93bd890c43fe4afa202af
  Stored in directory: /root/.cache/pip/wheels/cc/8d/09/563edfd874a35c0c7ed129b6c4fa890efa4c26458bdec6ffc1
Successfully built docx
     |████████████████████████████████| 5.5MB 5.6MB/s 
  Created wheel for python-docx: filename=python_docx-0.8.10-cp36-none-any.whl size=184491 sha256=013990303698711bfb1e1dc6ab5db3509476616bb96ed3fb7b28579df6af9145
  Stored in directory: /root/.cache/pip/wheels/18/0b/a0/1dd62ff812c857c9e487f27d80d53d2b40531bec1acecfa47b
Successfully built python-docx
  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=e68d81e45ee86b4e78c50c66beac4e74ecdbfdd45928791c4ed9783f0f279475
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Succes

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>



Now, please click the folder icon on the left-sidebar, right-click the file 2021_January_Milos.docx and select Download
